In [0]:
import json
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import tensorflow_hub as hub
from time import time
import io
import re
import csv
import ast
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from keras.utils import to_categorical
from keras.models import model_from_json

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support,classification_report
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras import backend as K
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.backend import sparse_categorical_crossentropy
from tensorflow.keras.layers import Dense, TimeDistributed
from tensorflow.python.tools import inspect_checkpoint as chkp

from sklearn.model_selection import train_test_split

from datetime import datetime

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
print(tf.__version__)

1.14.0


In [4]:
from google.colab import drive


drive.mount('/content/drive/')
__dir__ = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [12]:
!pip install bert-tensorflow

In [13]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

W0803 23:34:16.901755 140659012458368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [6]:
OUTPUT_DIR = '/content/drive/My Drive/Colab Notebooks/Model_Output/'
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: /content/drive/My Drive/Colab Notebooks/Model_Output/ *****


In [14]:
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1'

def create_tokenizer_from_hub_module():
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info =  bert_module(signature='tokenization_info', as_dict=True)
#     print(tokenization_info)
    with tf.compat.v1.Session() as sess:
      vocab_file = sess.run(tokenization_info['vocab_file'])
      
  return bert.tokenization.FullTokenizer(vocab_file=vocab_file)


tokenizer = create_tokenizer_from_hub_module()

W0803 23:34:30.036031 140659012458368 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/bert/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'be',
 '##rt',
 'token',
 '##izer']

In [0]:
def get_df(data_type):
  data_type_dict = {'train' : '/content/drive/My Drive/Colab Notebooks/data/onto.train.ner_bilou.csv',
                   'dev' : '/content/drive/My Drive/Colab Notebooks/data/onto.development.ner_bilou.csv',
                   'test' : '/content/drive/My Drive/Colab Notebooks/data/onto.test.ner_bilou.csv',
                   'bios': '/content/drive/My Drive/Colab Notebooks/data/bios-tagged-final-flat_bilou_no_title.csv'}

  df = pd.read_csv(data_type_dict[data_type])
  
  df['x'] = df['x'].apply(ast.literal_eval)
  X = df['x'].apply(' '.join)
  y = df['y'].apply(ast.literal_eval)
   
  return X, y

In [0]:
X_train, y_train = get_df('train')
X_dev, y_dev = get_df('dev')
X_test, y_test = get_df('test')
X_bios, y_bios = get_df('bios')

In [0]:
X_train[0]

'What kind of memory ?'

In [0]:
label_list = list(set([label for target_list in y_train for label in target_list])) + ['[CLS]', '[SEP]']
# print(label_list)
label_map = {label : i for i, label in enumerate(label_list,1)}
print(label_map)
print(f'Length of labels - {len(label_list)}')

{'U-PRODUCT': 1, 'I-QUANTITY': 2, 'U-PERCENT': 3, 'U-WORK_OF_ART': 4, 'L-PRODUCT': 5, 'B-ORG': 6, 'I-ORG': 7, 'U-ORG': 8, 'L-WORK_OF_ART': 9, 'B-PERCENT': 10, 'B-CARDINAL': 11, 'L-ORDINAL': 12, 'B-NORP': 13, 'B-GPE': 14, 'I-MONEY': 15, 'U-DATE': 16, 'L-LAW': 17, 'B-WORK_OF_ART': 18, 'L-NORP': 19, 'L-QUANTITY': 20, 'B-LANGUAGE': 21, 'L-LOC': 22, 'L-PERCENT': 23, 'B-PERSON': 24, 'U-ORDINAL': 25, 'L-PERSON': 26, 'U-MONEY': 27, 'B-LOC': 28, 'U-GPE': 29, 'I-DATE': 30, 'U-LANGUAGE': 31, 'L-DATE': 32, 'U-QUANTITY': 33, 'I-FAC': 34, 'B-TIME': 35, 'B-DATE': 36, 'I-PERSON': 37, 'I-ORDINAL': 38, 'L-CARDINAL': 39, 'I-CARDINAL': 40, 'I-WORK_OF_ART': 41, 'B-ORDINAL': 42, 'I-NORP': 43, 'U-TIME': 44, 'U-NORP': 45, 'L-EVENT': 46, 'B-QUANTITY': 47, 'I-LOC': 48, 'L-MONEY': 49, 'B-LAW': 50, 'L-FAC': 51, 'I-GPE': 52, 'U-PERSON': 53, 'U-CARDINAL': 54, 'L-ORG': 55, 'B-PRODUCT': 56, 'L-LANGUAGE': 57, 'B-MONEY': 58, 'B-FAC': 59, 'I-PERCENT': 60, 'U-FAC': 61, 'I-TIME': 62, 'U-LAW': 63, 'I-PRODUCT': 64, 'B-EVENT

In [0]:
label_map = {label : i for i, label in enumerate(label_list,1)}
print(label_map)

{'U-PRODUCT': 1, 'I-QUANTITY': 2, 'U-PERCENT': 3, 'U-WORK_OF_ART': 4, 'L-PRODUCT': 5, 'B-ORG': 6, 'I-ORG': 7, 'U-ORG': 8, 'L-WORK_OF_ART': 9, 'B-PERCENT': 10, 'B-CARDINAL': 11, 'L-ORDINAL': 12, 'B-NORP': 13, 'B-GPE': 14, 'I-MONEY': 15, 'U-DATE': 16, 'L-LAW': 17, 'B-WORK_OF_ART': 18, 'L-NORP': 19, 'L-QUANTITY': 20, 'B-LANGUAGE': 21, 'L-LOC': 22, 'L-PERCENT': 23, 'B-PERSON': 24, 'U-ORDINAL': 25, 'L-PERSON': 26, 'U-MONEY': 27, 'B-LOC': 28, 'U-GPE': 29, 'I-DATE': 30, 'U-LANGUAGE': 31, 'L-DATE': 32, 'U-QUANTITY': 33, 'I-FAC': 34, 'B-TIME': 35, 'B-DATE': 36, 'I-PERSON': 37, 'I-ORDINAL': 38, 'L-CARDINAL': 39, 'I-CARDINAL': 40, 'I-WORK_OF_ART': 41, 'B-ORDINAL': 42, 'I-NORP': 43, 'U-TIME': 44, 'U-NORP': 45, 'L-EVENT': 46, 'B-QUANTITY': 47, 'I-LOC': 48, 'L-MONEY': 49, 'B-LAW': 50, 'L-FAC': 51, 'I-GPE': 52, 'U-PERSON': 53, 'U-CARDINAL': 54, 'L-ORG': 55, 'B-PRODUCT': 56, 'L-LANGUAGE': 57, 'B-MONEY': 58, 'B-FAC': 59, 'I-PERCENT': 60, 'U-FAC': 61, 'I-TIME': 62, 'U-LAW': 63, 'I-PRODUCT': 64, 'B-EVENT

In [0]:
print(X_train[0])

What kind of memory ?


In [0]:
def conv_to_df(X, y):

  df = pd.DataFrame([X,y]).T
  df.columns = ['X', 'y']
#   print (type(df.loc[0, 'X']))
#   print (type(df.loc[0, 'y']))
#   df['y'] = df['y'].str.strip('[]')
#   print(df.head())
  return df

train = conv_to_df(X_train, y_train)
dev = conv_to_df(X_dev, y_dev)
test = conv_to_df(X_test, y_test)
bios = conv_to_df(X_bios, y_bios)

In [0]:
import logging
logger = logging.getLogger(__name__)

class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, valid_ids=None, label_mask=None):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.valid_ids = valid_ids
        self.label_mask = label_mask

def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {label : i for i, label in enumerate(label_list,1)}
    
    features = []
    for (ex_index,example) in enumerate(examples):
        textlist = example.text_a.split(' ')
        labellist = example.label
        if len(textlist) != len(labellist):
          continue
        else:
          tokens = []
          labels = []
          valid = []
          label_mask = []
          for i, word in enumerate(textlist): 
              token = tokenizer.tokenize(word)
              tokens.extend(token)

              label_1 = labellist[i]
              for m in range(len(token)):
                  if m == 0:
                      labels.append(label_1)
                      valid.append(1)
                      label_mask.append(1)
                  else:
                      valid.append(0)
          if len(tokens) >= max_seq_length - 1:
              tokens = tokens[0:(max_seq_length - 2)]
              labels = labels[0:(max_seq_length - 2)]
              valid = valid[0:(max_seq_length - 2)]
              label_mask = label_mask[0:(max_seq_length - 2)]
          ntokens = []
          segment_ids = []
          label_ids = []
          ntokens.append("[CLS]")
          segment_ids.append(0)
          valid.insert(0,1)
          label_mask.insert(0,1)
          label_ids.append(label_map["[CLS]"])
          for i, token in enumerate(tokens):
              ntokens.append(token)
              segment_ids.append(0)
              if len(labels) > i:
                  label_ids.append(label_map[labels[i]])
          ntokens.append("[SEP]")
          segment_ids.append(0)
          valid.append(1)
          label_mask.append(1)
          label_ids.append(label_map["[SEP]"])
          input_ids = tokenizer.convert_tokens_to_ids(ntokens)
          input_mask = [1] * len(input_ids)
          label_mask = [1] * len(label_ids)
          while len(input_ids) < max_seq_length:
              input_ids.append(0)
              input_mask.append(0)
              segment_ids.append(0)
              label_ids.append(0)
              valid.append(1)
              label_mask.append(0)
          while len(label_ids) < max_seq_length:
              label_ids.append(0)
              label_mask.append(0)
          assert len(input_ids) == max_seq_length
          assert len(input_mask) == max_seq_length
          assert len(segment_ids) == max_seq_length
          assert len(label_ids) == max_seq_length
          assert len(valid) == max_seq_length
          assert len(label_mask) == max_seq_length

          if ex_index < 5:
              logger.info("*** Example ***")
              logger.info("guid: %s" % (example.guid))
              logger.info("tokens: %s" % " ".join(
                      [str(x) for x in tokens]))
              logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
              logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
              logger.info(
                      "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
#               logger.info("label: %s (id = %d)" % (example.label, label_ids))

          features.append(
                  InputFeatures(input_ids=input_ids,
                                input_mask=input_mask,
                                segment_ids=segment_ids,
                                label_id=label_ids,
                                valid_ids=valid,
                                label_mask=label_mask))
    return features

In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

dev_InputExamples = dev.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

bios_InputExamples = bios.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['X'], 
                                                                   text_b = None, 
                                                                   label = x['y']), axis = 1)

In [0]:
MAX_SEQ_LENGTH = 56
max_length = MAX_SEQ_LENGTH

In [0]:
# Convert our train and test features to InputFeatures that BERT understands.
train_features = convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
dev_features = convert_examples_to_features(dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
test_features = convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
bios_features = convert_examples_to_features(bios_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
trainSentence_ids = []
trainMasks = []
trainSequence_ids = []

devSentence_ids = []
devMasks = []
devSequence_ids = []

testSentence_ids = []
testMasks = []
testSequence_ids = []

biosSentence_ids = []
biosMasks = []
biosSequence_ids = []

nerLabels_train =[]
nerLabels_dev = []
nerLabels_test = []
nerLabels_bios = []

bert_inputs_train = np.array([[f.input_ids for f in train_features], [f.input_mask for f in train_features],[f.segment_ids for f in train_features]])
labels_train = [f.label_id for f in train_features]
num_sentences_train = len([f.input_ids for f in train_features])
# print(num_sentences)
for example in range(num_sentences_train):
    trainSentence_ids.append(bert_inputs_train[0][example])
    trainMasks.append(bert_inputs_train[1][example])
    trainSequence_ids.append(bert_inputs_train[2][example])
    nerLabels_train.append(labels_train[example])

    
# labels_train = [to_categorical(f.label_id) for f in train_features]


bert_inputs_dev = [[f.input_ids for f in dev_features], [f.input_mask for f in dev_features],[f.segment_ids for f in dev_features]]
labels_dev = [f.label_id for f in dev_features]
num_sentences_dev = len([f.input_ids for f in dev_features])
for example in range(num_sentences_dev):
    devSentence_ids.append(bert_inputs_dev[0][example])
    devMasks.append(bert_inputs_dev[1][example])
    devSequence_ids.append(bert_inputs_dev[2][example])
    nerLabels_dev.append(labels_dev[example])

    
bert_inputs_test = np.array([[f.input_ids for f in test_features], [f.input_mask for f in test_features],[f.segment_ids for f in test_features]])
labels_test = [f.label_id for f in test_features]
num_sentences_test = len([f.input_ids for f in test_features])
# print(num_sentences)
for example in range(num_sentences_test):
    testSentence_ids.append(bert_inputs_test[0][example])
    testMasks.append(bert_inputs_test[1][example])
    testSequence_ids.append(bert_inputs_test[2][example])
    nerLabels_test.append(labels_test[example])    
    
    
bert_inputs_bios = np.array([[f.input_ids for f in bios_features], [f.input_mask for f in bios_features],[f.segment_ids for f in bios_features]])
labels_bios = [f.label_id for f in bios_features]
num_sentences_bios = len([f.input_ids for f in bios_features])
# print(num_sentences)
for example in range(num_sentences_bios):
    biosSentence_ids.append(bert_inputs_bios[0][example])
    biosMasks.append(bert_inputs_bios[1][example])
    biosSequence_ids.append(bert_inputs_bios[2][example])
    nerLabels_bios.append(labels_bios[example])    
    
X_train = np.array([trainSentence_ids,trainMasks,trainSequence_ids])
X_dev = np.array([devSentence_ids,devMasks,devSequence_ids])
X_test = np.array([testSentence_ids,testMasks,testSequence_ids])
X_bios = np.array([biosSentence_ids,biosMasks,biosSequence_ids])

nerLabels_train = np.array(nerLabels_train)
nerLabels_dev = np.array(nerLabels_dev)
nerLabels_test = np.array(nerLabels_test)
nerLabels_bios = np.array(nerLabels_bios)

In [0]:
X_bios.shape

(3, 708, 56)

In [0]:
k_start = 0
k_end = -1

if k_end == -1:
    k_end_train = X_train[0].shape[0]
    k_end_dev = X_dev[0].shape[0]
    k_end_test = X_test[0].shape[0]
    k_end_bios = X_bios[0].shape[0]
else:
    k_end_train = k_end_dev = k_end
    


bert_inputs_train_k = [X_train[0][k_start:k_end_train], X_train[1][k_start:k_end_train], 
                       X_train[2][k_start:k_end_train]]
bert_inputs_dev_k = [X_dev[0][k_start:k_end_dev], X_dev[1][k_start:k_end_dev], 
                      X_dev[2][k_start:k_end_dev]]
bert_inputs_test_k = [X_test[0][k_start:k_end_test], X_test[1][k_start:k_end_test], 
                      X_test[2][k_start:k_end_test]]
bert_inputs_bios_k = [X_bios[0][k_start:k_end_test], X_bios[1][k_start:k_end_test], 
                      X_bios[2][k_start:k_end_test]]

labels_train_k = nerLabels_train[k_start:k_end_train]
labels_dev_k = nerLabels_dev[k_start:k_end_dev]
labels_test_k = nerLabels_test[k_start:k_end_test]
labels_bios_k = nerLabels_bios[k_start:k_end_bios]

In [0]:
for item in bert_inputs_train_k:
  print(item.shape)

(115812, 56)
(115812, 56)
(115812, 56)


In [0]:
class BertLayer(tf.keras.layers.Layer):
    """
    Create BERT layer, following https://towardsdatascience.com/bert-in-keras-with-tensorflow-hub-76bcbc9417b
    init:  initialize layer. Specify various parameters regarding output types and dimensions. Very important is
           to set the number of trainable layers.
    build: build the layer based on parameters
    call:  call the BERT layer within a model
    """
    
    def __init__(
        self,
        n_fine_tune_layers=10,
        pooling="sequence",
        bert_url="https://tfhub.dev/google/bert_cased_L-12_H-768_A-12/1",
        **kwargs,
    ):
        self.n_fine_tune_layers = n_fine_tune_layers
        self.trainable = True
        self.output_size = 768
        self.pooling = pooling
        self.bert_url = bert_url

        super(BertLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.bert = hub.Module(
            self.bert_url, trainable=self.trainable, name=f"{self.name}_module"
        )

        # Remove unused layers
        trainable_vars = self.bert.variables
        trainable_vars = [
                var
                for var in trainable_vars
                if not "/cls/" in var.name and not "/pooler/" in var.name
            ]
        trainable_layers = []


        # Select how many layers to fine tune
        for i in range(self.n_fine_tune_layers):
            trainable_layers.append(f"encoder/layer_{str(11 - i)}")

        # Update trainable vars to contain only the specified layers
        trainable_vars = [
            var
            for var in trainable_vars
            if any([l in var.name for l in trainable_layers])
        ]

        # Add to trainable weights
        for var in trainable_vars:
            self._trainable_weights.append(var)

        for var in self.bert.variables:
            if var not in self._trainable_weights:
                self._non_trainable_weights.append(var)

        super(BertLayer, self).build(input_shape)

    def call(self, inputs):
        inputs = [K.cast(x, dtype="int32") for x in inputs]
        input_ids, input_mask, segment_ids = inputs
        bert_inputs = dict(
            input_ids=input_ids, input_mask=input_mask, segment_ids=segment_ids
        )
        result = self.bert(inputs=bert_inputs, signature="tokens", as_dict=True)[
                "sequence_output"
            ]

        mul_mask = lambda x, m: x * tf.expand_dims(m, axis=-1)

        return result

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_size)

In [0]:
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)


In [0]:


def custom_loss(y_true, y_pred):
    """
    calculate loss function explicitly, filtering out 'extra inserted labels'
    
    y_true: Shape: (batch x (max_length + 1) )
    y_pred: predictions. Shape: (batch x x (max_length + 1) x num_distinct_ner_tokens ) 
    
    returns:  cost
    """

    #get labels and predictions
    
    y_label = tf.reshape(tf.layers.Flatten()(tf.cast(y_true, tf.int32)),[-1])
#     print(y_label.eval())
    mask = (y_label < 74)   # This mask is used to remove all tokens that do not correspond to the original base text.

    y_label_masked = tf.boolean_mask(y_label, mask)  # mask the labels
    
    y_flat_pred = tf.reshape(tf.layers.Flatten()(tf.cast(y_pred, tf.float32)),[-1, 75])
#     print(y_flat_pred.eval())
    y_flat_pred_masked = tf.boolean_mask(y_flat_pred, mask) # mask the predictions
    
    return tf.reduce_mean(sparse_categorical_crossentropy(y_label_masked, y_flat_pred_masked,from_logits=False ))

In [0]:
def custom_acc_orig_tokens(y_true, y_pred):
    """
    calculate loss dfunction filtering out also the newly inserted labels
    
    y_true: Shape: (batch x (max_length) )
    y_pred: predictions. Shape: (batch x x (max_length + 1) x num_distinct_ner_tokens ) 
    
    returns: accuracy
    """

    #get labels and predictions
    
    y_label = tf.reshape(tf.layers.Flatten()(tf.cast(y_true, tf.int64)),[-1])
    
    mask = (y_label < 74)
    y_label_masked = tf.boolean_mask(y_label, mask)
    
    y_predicted = tf.math.argmax(input = tf.reshape(tf.layers.Flatten()(tf.cast(y_pred, tf.float64)),\
                                                    [-1, 75]), axis=1)
    
    y_predicted_masked = tf.boolean_mask(y_predicted, mask)

    return tf.reduce_mean(tf.cast(tf.equal(y_predicted_masked,y_label_masked) , dtype=tf.float64))

In [0]:
def custom_acc_orig_non_other_tokens(y_true, y_pred):
    """
    calculate loss dfunction explicitly filtering out also the 'Other'- labels
    
    y_true: Shape: (batch x (max_length) )
    y_pred: predictions. Shape: (batch x x (max_length + 1) x num_distinct_ner_tokens ) 
    
    returns: accuracy
    """

    #get labels and predictions
    
    y_label = tf.reshape(tf.layers.Flatten()(tf.cast(y_true, tf.int64)),[-1])
    
    mask = (y_label < 74)
    y_label_masked = tf.boolean_mask(y_label, mask)
    
    y_predicted = tf.math.argmax(input = tf.reshape(tf.layers.Flatten()(tf.cast(y_pred, tf.float64)),\
                                                    [-1, 75]), axis=1)
    
    y_predicted_masked = tf.boolean_mask(y_predicted, mask)

    return tf.reduce_mean(tf.cast(tf.equal(y_predicted_masked,y_label_masked) , dtype=tf.float64))

In [0]:
adam_customized = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False)

In [0]:


def ner_model(max_input_length, train_layers, optimizer):
    """
    Implementation of NER model
    
    variables:
        max_input_length: number of tokens (max_length + 1)
        train_layers: number of layers to be retrained
        optimizer: optimizer to be used
    
    returns: model
    """
    
    in_id = tf.keras.layers.Input(shape=(max_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_length), name="segment_ids")
    
    
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_sequence = BertLayer(n_fine_tune_layers=train_layers)(bert_inputs)
    
    print(bert_sequence)
    
    dense = tf.keras.layers.Dense(256, activation='relu', name='dense')(bert_sequence)
    
#     dense = tf.keras.layers.Dropout(rate=0.2)(dense)
    
    pred = tf.keras.layers.Dense(75, activation='softmax', name='ner')(dense)
     
    print('pred: ', pred)
    
    ## Prepare for multipe loss functions, although not used here
    
    losses = {
        "ner": custom_loss,
        }
    lossWeights = {"ner": 1.0
                  }
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs=pred)

#     model.compile(loss=losses, optimizer=optimizer, metrics=[custom_acc_orig_tokens, 
#                                                           custom_acc_orig_non_other_tokens])
    
    model.compile(loss=custom_loss, optimizer=optimizer, metrics=[custom_acc_orig_tokens, 
                                                          custom_acc_orig_non_other_tokens])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)


In [0]:
sess = tf.Session()

model = ner_model(max_length , train_layers=10, optimizer = adam_customized)

# Instantiate variables
initialize_vars(sess)



# tensorboard = TensorBoard(log_dir="logs/{}".format(time()))

model.fit(
    bert_inputs_train_k, 
    {"ner":labels_train_k},
    validation_data=(bert_inputs_dev_k, {"ner":labels_dev_k}),
    epochs=3,
    batch_size=50#,
    #callbacks=[tensorboard]
)

Tensor("bert_layer_1/bert_layer_1_module_apply_tokens/bert/encoder/Reshape_13:0", shape=(?, ?, 768), dtype=float32)
pred:  Tensor("ner_1/truediv:0", shape=(?, ?, 75), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 56)]         0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 56)]         0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 56)]         0                                            
__________________________________________________________________________________________________
bert_laye

In [0]:
with open(OUTPUT_DIR + 'bert_bilou_architecture.json', 'w') as f:
    f.write(model.to_json())

model.save_weights(OUTPUT_DIR + 'bert_bilou_model_weights.h5')

In [0]:
# json_file = open(OUTPUT_DIR + 'bert_bilou_architecture.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# loaded_model.load_weights("bert_bilou_model_weights.h5")
# print("Loaded model from disk")

In [0]:
def reshape_predictions(yh, pr):
#     coords = [np.where(yhh > 0)[0] for yhh in yh]
    yh = [yhh for yhh, co in zip(yh, coords)]
    ypr = [prr for prr, co in zip(pr, coords)]
#     fyh = [c for row in yh for c in row]
#     fpr = [c for row in ypr for c in row]
    return yh, ypr

In [0]:
#     bert_inputs_train_k, 
#     {"ner":labels_train_k}
y_pred = model.predict(bert_inputs_test_k).argmax(2)
yh = labels_test_k


In [0]:
fyh_test, fpr_test = y_pred.flatten(), yh.flatten()

In [0]:
score = model.evaluate(bert_inputs_test_k, labels_test_k)
print(f'Model loss: {score}')

In [0]:
print('Training accuracy Onto TEST : ', accuracy_score(y_pred.flatten(), yh.flatten()))
# print('Training confusion matrix:')
cm = confusion_matrix(fyh_test, fpr_test)

Training accuracy Onto TEST :  0.8497023809523809


In [0]:
print(y_pred[0])
print(labels_test_k[0])

[72 72 72 72 72 72 72 72 72 72 72 72 72 72 72 72  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[74 72 72 72 72 72 72 72 72 72 72 72 72 72 72 75  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


In [0]:
print(classification_report(fyh_test, fpr_test))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           0       1.00      0.98      0.99    442409
           1       0.00      0.00      0.00         0
           2       0.08      0.50      0.14        22
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.23      0.58      0.33       456
           7       0.41      0.50      0.45      1251
           8       0.18      0.69      0.28       218
           9       0.00      0.00      0.00         0
          10       0.39      0.52      0.44       300
          11       0.17      0.60      0.26        60
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.16      0.52      0.25       154
          15       0.56      0.61      0.58       404
          16       0.20      0.68      0.31       200
          17       0.00      0.00      0.00         0
          18       0.00    

In [25]:
y_pred = model.predict(bert_inputs_bios_k).argmax(2)
yh = labels_bios_k
fyh_bios, fpr_bios = y_pred.flatten(), yh.flatten()


NameError: ignored

In [0]:
score_bios = model.evaluate(bert_inputs_bios_k, labels_bios_k)
print(f'Model loss: {score_bios}')

In [24]:
print('Training accuracy DEF14A : ', accuracy_score(fyh_bios, fpr_bios))
# print('Training confusion matrix:')
cm = confusion_matrix(fyh_test, fpr_test)

NameError: ignored

In [0]:
print(classification_report(fyh_bios, fpr_bios))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98     23246
           1       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.13      0.57      0.22       199
           7       0.31      0.47      0.38       796
           8       0.07      0.57      0.13        28
          10       0.50      1.00      0.67         1
          13       0.00      0.00      0.00         0
          14       0.14      0.60      0.23         5
          15       1.00      0.50      0.67         4
          16       0.07      0.41      0.12        37
          17       0.00      0.00      0.00         0
          19       0.00      0.00      0.00         0
          23       0.50      1.00      0.67         1
          24       0.86      0.83      0.85       103
          25       0.00      0.00      0.00         2
          26       0.55      0.50      0.52       111
          29       0.17    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
